In [1]:
from typing import (
    Optional,
    Tuple,
    Union,
)

import ipytest
import numpy as np
import pytest
import torch
from torch import nn
from torch.nn.common_types import Tensor

In [2]:
ipytest.autoconfig()

In [94]:
def upsample(
        input: Tensor, 
        size: Optional[Union[int, Tuple[int, int]]] = None,
        scale_factor: Optional[Union[float, Tuple[float, float]]] = None, 
        mode: str = 'bilinear',   
    ):
    if mode != 'bilinear':
        raise ValueError("Параметр mode может быть только bilinear")
    
    if not size and not scale_factor:
        raise ValueError("Заполните один из параметров: size, scale_factor")
    
    batch_size, channels, input_height, input_width = input.shape

    if size:
        (result_height, result_width) = (size, size) if isinstance(size, int) else size
        scale_factor = (result_height / input_height, result_width / input_width)
    elif scale_factor:
        scale_factor = (scale_factor, scale_factor) if isinstance(scale_factor, float) else scale_factor
        result_height, result_width = int(input_height*scale_factor[0]), int(input_width*scale_factor[1])

    result = np.zeros((batch_size, channels, result_height, result_width))

    r = (np.arange(result_height) + 0.5) / scale_factor[0] - 0.5
    c = (np.arange(result_width) + 0.5) / scale_factor[1] - 0.5

    for i in range(result_height):
        for j in range(result_width):
            row = r[i]
            col = c[j]
            r0 = int(row)
            r1 = min(r0 + 1, input_height - 1)
            c0 = int(col)
            c1 = min(c0 + 1, input_width - 1)
            dr = row - r0
            dc = col - c0

            for k in range(channels):
                result[:, k, i, j] = (1 - dr) * (1 - dc) * input[:, k, r0, c0] + dr * (1 - dc) * input[:, k, r1, c0] + (1 - dr) * dc * input[:, k, r0, c1] + dr * dc * input[:, k, r1, c1]

    return result

In [95]:
@pytest.fixture(scope='class')
def inputs():
    return torch.randn(2, 4, 1, 1)

In [101]:
%%ipytest -s

@pytest.mark.usefixtures('inputs')
class TestConv2D:
    def test_upscaling_scale_factor_success(self, inputs):
        result = upsample(inputs, scale_factor=6.0)
        expected_result = nn.functional.upsample(inputs, scale_factor=6, mode='bilinear') .numpy()
        assert np.allclose(expected_result, result, atol=1e-4, rtol=1e-3)
        
    def test_upscaling_size_success(self, inputs):
        result = upsample(inputs, size=4)
        expected_result = nn.functional.upsample(inputs, size=4, mode='bilinear') .numpy()
        assert np.allclose(expected_result, result, atol=1e-4, rtol=1e-3)

.

.
======================================== warnings summary =========================================
t_b1742cceadd4476abdc5ac6046714271.py::TestConv2D::test_upscaling_scale_factor_success
t_b1742cceadd4476abdc5ac6046714271.py::TestConv2D::test_upscaling_size_success
  c:\Python311\Lib\site-packages\torch\nn\functional.py:3769: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
    warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
2 passed, 2 warnings in 0.04s
